## Collaborative Filtering with Alternating Least Squares

### Introduction

    In 2016, more than half of all Americans consumed music through online streaming services on a weekly basis. Online streaming services generated $17.3 billion in revenue in 2017, up 8.1% from 2016. As more listening moves from mp3, AM/FM and physical mediums to streaming, online radio firms are relying on machine learning to rapidly personalize music selection and discovery for users in order to gain a competitive advantage in growing marketplace. Companies such as Spotify and Google have purchased smaller technology start-ups for their audio machine learning solutions (EchoNest) or their extensive customer data (Songza) to get leg up in this digital arms race.
          Streaming companies utilize a variety of methods to recommend music to their users. Pandora allows users to give songs  a “thumbs up or down” to indicate positive or negative preference. This explicit feedback helps Pandora quickly create personalized playlists, where songs with similar attributes are served to the user based off their rating feedback. Spotify generates playlists based off your listeneing history and factors in everything from time of day to location. With machine learning, a streaming company can use a variety of data to infer a listener’s taste and recommend songs the user is more likely to prefer automatically. Using recommendation systems, companies can create personalized playlists and radio stations without employing taste-makers or experts. 

### Recommendation Systems

        Recommendation systems can be broken into two categories: content-based and collaborative filters. Content-based systems rely on attributes related to the item/product (a song or artist in this case) to make recommendations to a user based off known user data. While this apprach is effective, it requres storing a large amount of data about the users and items. Often times, companies don't always have customer data to create this type of recommendation system.
    
        A collaborative filter creates a recommendation engine using a user identifier, an item identifier and some rating metric that shows an interaction between the user and item. Based off a collection these user/item interactions, the collaborative filter can predict a user's potential item preference based off users with similar tastes. Similarly, a collaborative filter can infer item similarities as well. 
        Collaborative filters can deal with two types ratings data: explicit or implicit. Explicit data means a user has directly made their preference known--similar to Pandora's thumbs up or thumbs down or a 5-star rating system like Yelp's restaurant reviews. Implicit ratings are a feedback metric does not necessarily give us a user's preference, but we can potentially infer preference--like a product purchase, time spent on a webpage or a song play. A using song plays as an example, a single listen to a song does not necessarily mean the user likes that song, but the more times a user listens to that same song, we can more confidently infer the user has a positive preference for said song. While implicit metrics sacrifice a degree of certainty, implict data is much easier to come by in the real world. 
    
    
### The Task

        The Million Song Challenge (MSC) data set is ideal for creating a recommendation system. Moreover, its format allows us to create a collaborative filter using implicit data. The dataset itself contains over 1 million unique user’s listening data and over 300,000 songs. The task is to create a robust recommendation system with just the interaction between users and songs. As a basis of comparison, we will be comparing the performance of our recommender model to serving the most popular song any given user has not listened to. This is a high bar to clear, as recommending the most popular item tends have a higher accuracy than a recommendation system
       
    Given the structure of the dataset, which is sparse--as most users will have only listened to several songs, and we are dealing with implicit ratings data, I will use an Alternating Least Squares (ALS) algorithm with matrix factorization. 
    
### Matrix Factorization

    Matrix factorization is a form of latent factor analysis. We start with a sparse matrix M which consists of users * items with ratings metric (song plays) in the cells. With matrix factorization, we factor matrix M into two separate matrices, X and Y; these matrices contain latent factors which reduce large numbers of interactions between users and items into several hidden or unobserved reasons. The X matrix is the users * these hidden features while the Y matrix is the items * hidden factors. Both matrices have a observations which  correspond with either a user or feature from the original matrix M. We then take the product of these matrices to fill in the missing entries of M to provide the liklihood of a user prefering an item.
    
$$M = XY^T$$
    
    The ALS algorithm is used to find, or approximate, X and Y. Both X and Y aren't known, so values are randomly initialized, first for Y. Then X is solved given the random values of Y and M. Using linear algebra, the algorithm solves for each row of X (i).
    
$$M_1Y(Y^TY)^{-1} = X_i$$

    Since the algorithm will not arrive at the exact equivalent, since X and Y are too small/low rank to equal M, values are optimized by minimizing the sum of squared differences between the both matrix entries. 
    
    In an implicit scenario, our original matrix M is converted to a binary preference matrix, matrix B. Matrix M is kept to incorporate the positive values as weights. For example, a user who listened to a song 100 times will be said to have more preference than someone who listened to a song 2 times.




In [3]:
# Import the relevent libraries

import numpy as np
import pandas as pd
import scipy.sparse as sp
import random
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
import implicit
import random
from sklearn import metrics

In [4]:
# Import the training and test data

# Training data
train = pd.read_csv('train_triplets.txt', delimiter = '\t', header = None)
train.columns = ['users', 'songs', 'play count']

# Test data
test = pd.read_csv('kaggle_visible_evaluation_triplets.txt', delimiter = '\t', header = None, names = ['users', 'songs', 'play count'])


FileNotFoundError: File b'train_triplets.txt' does not exist

### Train-Test Split?

    Typical machine learning problems involve training an algorithm on a one dataset and then testing the model on a hold-out dataset. This approach will not work here, as we need all possible user-item interactions to optimize our latent factors vectors. We want to reduce the amount of sparsity as much as possible. Instead we will mask a certain percentage of user-item interactions and see if our algorithm correctly predicted if a user preferred that song.



In [ ]:
# combining the two dataframe

df = pd.concat([train, test], ignore_index = True)


### Track information 

    There is a separate .txt file which maps the song ID from the main dataset to the actual song name and artist name. This information will be useful later when analyzing the results of the data. Viewing the recommendations as songIDs will not be as intuitive as viewing them as a collection of recongnizable songs. 

In [5]:
# import track and artist information for later analysis

track_info = pd.read_csv('unique_tracks.txt', header = None, delimiter = '\<SEP>', names = ['track_id', 'songs', 'artist', 'song_title'] )

# drop duplicates by songID

track_info_unique = track_info.drop_duplicates(subset = ['songs'])

track_info.song_title = track_info['song_title'].astype(str) + "-" + track_info['artist'].astype(str)

# merge dataframe

df_track = df.merge(track_info_unique[['songs', 'song_title']], on = 'songs', how = 'inner')

# check first and last 5 entries of new dataframe

df_track.head()
df_track.tail()



C:\Users\sawal\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


FileNotFoundError: [Errno 2] No such file or directory: 'unique_tracks.txt'

### Exploratory Data Analysis

    Unlike more traditional machine learning tasks, the exploratory data analysis phase is limited, since there are relatively few fields. There is still value in performing basic exploration. 

In [ ]:
df_track['play_count'].describe()

# How many unique users

print(len(df_track['users'].drop_duplicates()))

# How many unique songs

print(len(df_track['songs'].drop_duplicates()))

# Check for null values

print(df_track['play count'].isnull().sum())

# Check the max and min of each column

play_count_max = df_track['play count'].max()
play_count_min = df_track['play count'].min()


In [ ]:
# pivot data to determine most popular songs

song_pivot = df_track.pivot_table(index = 'song_title',
                      aggfunc = sum)
print(song_pivot[25])


# Check the most popular songs and artist

song_popularity = df_track.song_title.value_counts()[25]

artist_popularity = df_track.artist.value_counts()[25]

# Check distribution

print(df_track['play count'].quantile(np.arange(0,1,0.05))


In [ ]:
# Plot histogram for distribution
_ = plt.hist(df_track['play count'], bins = 10, range = [play_count_min, 40], normed = True)
_ = plt.xlabel('Play counts')
_ = plt.ylabel('Counts')
plt.show()

# Plot distplot

_ = sns.distplot(df_track['artist'])
plt.show()

### Data Preprocessing

    The ALS algorithm requires that our data be in the form of a sparse matrix of users * items with interactions (play count) in the cells. Therefore, we will create a sparse matrix with the relevent columns. We must also encode the user and item variables for faster processing. We will map these codes to the song title and artist name for interpreting our model at a later point. 

In [ ]:
# Encode relevent variables


df_track['user_id'] = df_track['users'].astype("category").cat.codes
df_track['song_id'] = df_track['songs'].astype("category").cat.codes

# Create lookup for songs and artists based off eoncodes variables

song_map = df_track[['songs', 'song_title']].drop_duplicates()
song_map['songs'] = song_map['songs'].astype(str)

In [ ]:
# Create sparse matrix

sp_item_user = sp.csr_matrix((df_track['play count'].astype(float), (df_track['user_id'], df_track['song_id'])))
sp_user_item = sp.csr_matrix((df_track['play count'].astype(float), (df_track['song_id'], df_track['user_id'])))

# Calculate density

sparsity = 1-(df_track.shape[0] / (df_track.user_id.unique().shape[0] * df_track.song_id.unique().shape[0]))
print(sparsity)


### Testing Recommendation Efficacy

    As previously mentioned, using a traditional test-train split will not work for our purposes. We will use a function that masks a certain percentage of user-item interactions on a validation set of data. After creating our recommender, we will check to see if our model ended up serving songs the user already listened to in the masked entry.

In [ ]:
def train_test_mask(sp_matrix, test_split = 0.20):
  
    val_set = sp_matrix.copy() # make test set from train data 
    val_set[val_set != 0] = 1 # Create unary matrix, all interactions
    train = sp_matrix.copy() # train set where certain interactions will be hidden
    nonzero_inds = train.nonzero() # use nonzero method to store user/items with interactions
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # list of user/song interactions that aren't zero
    random.seed(123) # set random seed for reproducability
    mask_number = int(np.rint(test_split*len(nonzero_pairs))) #  # of samples
    masked = random.sample(nonzero_pairs, mask_number) # Sample a random number of user-item pairs (no replacement)
    user_inds = [index[0] for index in masked] # All user row indices
    item_inds = [index[1] for index in masked] # All song column indices
    train[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero thereby masking them
    train.eliminate_zeros() # Delete zeros in sparse array storage after update to save space
    return train, val_set, list(set(user_inds)) # 

train, val_set, user_list_train = train_test_mask(sp_item_user)

### Creating the model

    A python package called Implicit is a great out-of-the-box option for creating a collaborative filter using the ALS method. This package uses Cython for processing code with different threads and the syntax is very straightforward. The hyperparameters are alpha, which is a scaling value for our ratings matrix, regularization, an overfitting parameter, number of factors, the number of hidden or latent factors and iterations, which is the number of times to switch between the user and item matrices in the alternating least squares process. We used the basic parameters here (hyperparamter tuning is restricted here due to memory constraints--see next section in Spark)

In [ ]:
# Use alternating Least Squares to create recommendation algorithm

alpha = 15
rec = implicit.als.AlternatingLeastSquares(factors = 50, regularization = 0.1,
                                           iterations = 20)
rec.fit(train * alpha)


user_vec, item_vec = rec.item_factors, rec.user_factors.transpose()


# Spot check recommendations for a user

# Choose user

user = df_track['user_id'][0]

# Function to return a dataframe of song recommendations

def user_recommendations(user, sp_matrix):
    songs = []
    scores = []
    recommended_songs = rec.recommend(user, sp_matrix)
    for song in recommended_songs:
        idx, score = song
        songs.append(df_track.song_title.loc[df_track.song_id == idx].iloc[0])
        scores.append(score)
        
        recommendations = pd.DataFrame({'songs': songs, 'score': score})
    return recommendations

user_recommdations(user, sp_user_item)
        

### Testing Recommendation Efficacy continued



In [ ]:
# Area under the curve (AUC) score function

def auc_score(pred, actual):

    fpr, tpr, thresholds = metrics.roc_curve(actual, pred)
    return metrics.auc(fpr, tpr)  



# Creating ROC curve and evaluating our model against a naive model which serves the most popular song the user has not previously listened to
# by comparing the AUC scores

def area_under_curve(training_set, user_list, predictions, val_set):

    
    als_auc = [] # list for item that
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(val_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for index, user in enumerate(user_list): # Iterate through each user that had an item altered
        print('Loop: {}'.format(index) + ' out of {}'.format(len(user_list))) # Note which part of the loop we are on (this will take a while)
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = val_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # take the unary 1/0 interaction pairs from the original matrix
        # which align with the same pairs in train set 
        pop = pop_items[zero_inds] # Get most popular item
        als_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular
        
    
    return float('%.3f'%np.mean(als_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC for our model and the most popular item

area_under_curve(train, user_list_train, 
              [sp.csr_matrix(user_vec), sp.csr_matrix(item_vec)], val_set)


### Tuning our Model

    Our model did not outperform serving the most popular songs but still scored high. As mentioned in the beginning, popularity based models tend to outperform basic recommendation systems. In order to tune our model further, we will reduce the sparsity of our model by removing users and items below a certain threshold of interaction. A rule of thumb when dealing with sparse data in a recommender system is we do not want sparsity to exceed 99.5%. Therefore we will remove all users who have listened below x number of songs and songs not listened to at least x number of times until our model reaches meets this threshold 

In [ ]:
def activity_thresh(df, user_min, song_min):
    while True:
        start_dim = df.shape[0]
        song_counts = df.groupby('user_id').song_id.count()
        df = df[~df.user_id.isin(song_counts[song_counts < song_min].index.tolist())]
        user_counts = df.groupby('song_id').user_id.count()
        df = df[~df.song_id.isin(user_counts[user_counts < user_min].index.tolist())]
        end_dim = df.shape[0]
        if start_dim == end_dim:
            break
    
    n_users = df.user_id.unique().shape[0]
    n_items = df.song_id.unique().shape[0]
    sparsity = 1- (float(df.shape[0]) / float(n_users*n_items))
    print('Number of users: {}'.format(n_users))
    print('Number of songs: {}'.format(n_items))
    print('Sparsity: {:.5%}'.format(sparsity))
    return df, sparsity


for i in range(5,50):
    print('Min user and song count: {}'.format(i))
    _, sparsity = activity_thresh(df, i, i)
    if sparsity < 0.995:
        break
    else:
        continue
        
# 25 is the optimal number


df_red, _ = activity_thresh(df, 25, 25)


In [ ]:
# Create item/user and user/item sparse matrices
sp_item_user = sp.csr_matrix((df_red['play count'].astype(float), (df_red['user_id'], df_red['song_id'])))
sp_user_item = sp.csr_matrix((df_red['play count'].astype(float), (df_red['song_id'], df_red['user_id']))



train_2, val_set_2, user_list_train_2 = train_test_mask(sp_item_user)


In [ ]:
# Use alternating Least Squares to create recommendation algorithm
alpha = 15
rec = implicit.als.AlternatingLeastSquares(factors = 50, regularization = 0.1,
                                           iterations = 20)
rec.fit(song_train * alpha)


user_vec, item_vec = rec.item_factors, rec.user_factors.transpose()

# check that these two dimensions match


train, val_set, user_list_train   


area_under_curve(train_2, user_lst_train_2, 
              [sp.csr_matrix(user_vec), sp.csr_matrix(item_vec)], val_set_2)



### Final Thoughts -- Next Steps

    Our AUC score improved but still did not exceed the popularity score. To improve our model further, we will need to perform hyperparameter tuning. However, this requires more performance than a standard desktop allows for--running the AUC curve function took over a day on my machine. We will perform the rest of this project using Spark. Spark's distributed nature allows for much better performance

References:

- Recommender Systems; Charu C. Aggarwal; 2016
- Advanced Analytics with Spark; Sean Owen; 2017
- http://yifanhu.net/PUB/cf.pdf
- https://jessesw.com/Rec-System/
- https://github.com/benfred/implicit

    
    